In [ ]:
# INSTALL AND IMPORT LIBRARIES
if (("boot" %in% installed.packages()[,"Package"]) == FALSE) install.packages("boot")
library("boot")

if (("pROC" %in% installed.packages()[,"Package"]) == FALSE) install.packages("pROC")
library("pROC")

In [ ]:
# IMPORT PROBABILISTIC PREDICTIONS TO BE BOOTSTRAPPED
for (file in list.files()) {
    if (length(grep("probabilistic_prediction",file)>0)) {
        assign(gsub(".csv","",file), as.numeric(unlist(read.csv(file, header = FALSE))))
    }
}

# IMPORT TRUE PREDICTIONS 
assign("true_outcome_complete", as.numeric(unlist(read.csv("true_outcome_complete.csv", header = FALSE))))

In [ ]:
# BOOSTRAP

auc_func_this <- function(data){
    auc(response = true_outcome_complete[data], predictor = probabilistic_prediction_complete_wra[data])
}

AUC_CI_BOOSTRAP <- data.frame(MODEL = "", LOW_CI = 0, HIGH_CI = 0)[-1,]
AUC_CI_BOOSTRAP$MODEL <- as.character(AUC_CI_BOOSTRAP$MODEL)

for (object in ls()) {
    if (length(grep("probabilistic_prediction",object)>0)) {
        auc_func_this <- function(original, data){
                        auc(response = true_outcome_complete[data], predictor = get(object)[data])
        }
        CI_THIS <- boot.ci(boot(1:length(true_outcome_complete), 
                                auc_func_this, 
                                10000, 
                                stype='i',
                               strata = true_outcome_complete),
                           conf = 0.95,
                           type = "bca")
        # print(CI_THIS)
        AUC_CI_BOOSTRAP[nrow(AUC_CI_BOOSTRAP)+1,"MODEL"] <- object
        AUC_CI_BOOSTRAP[nrow(AUC_CI_BOOSTRAP),"LOW_CI"] <- CI_THIS$bca[4]
        AUC_CI_BOOSTRAP[nrow(AUC_CI_BOOSTRAP),"HIGH_CI"] <- CI_THIS$bca[5]
        
    }
}



In [ ]:
# cleansing
AUC_CI_BOOSTRAP$MODEL <- gsub("_complete_","",gsub("_test_UNIVARIATE_FEATURE_IMPORTANCE","",gsub("probabilistic_prediction","",AUC_CI_BOOSTRAP$MODEL)))

In [ ]:
# EXPORT RESULTS
write.csv(AUC_CI_BOOSTRAP, "CI_BOOSTRAP_AUCS.csv",col.names=TRUE, row.names=FALSE)